In [1]:
# imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import requests
import pickle
import io

In [2]:
# path to the dataset (replace with your own path) MAKE SURE YOU CHANGE THE SLASHES
path = "C:/Users/Asa/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1"

In [12]:
import preprocessing as prep
target_size = (150, 150)
grayscale = True
X, y = prep.get_X_y(path=path, percent=0.5, target_size=target_size, grayscale=grayscale)

In [13]:
no_output_classes = len(y.columns)

In [14]:
y[:5]

,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
X.shape

(2608, 150, 150)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
print("Train Lengths:", len(X_train), len(y_train))
print("Test Lengths:", len(X_test), len(y_test))

Train Lengths: 1956 1956
Test Lengths: 652 652


In [19]:
# defining the model
from tensorflow.keras import layers

input_shape = target_size
if grayscale:
    input_shape = input_shape + (1,)
else:
    input_shape = input_shape + (3,)

# removing images from the dataset
# dropout: randomly remove nodes in each layer to create gaps
# normalization layer
# augmentation
# convert to grayscale <- 
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # added for improvement HOPEFULLY! :)
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(no_output_classes, activation='sigmoid')
])

c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [21]:
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 24s 331ms/step - accuracy: 0.1392 - loss: 2.0644 - val_accuracy: 0.2623 - val_loss: 1.9138
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 327ms/step - accuracy: 0.2985 - loss: 1.8294 - val_accuracy: 0.3543 - val_loss: 1.6978
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 19s 314ms/step - accuracy: 0.3779 - loss: 1.7162 - val_accuracy: 0.3589 - val_loss: 1.7150
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 314ms/step - accuracy: 0.4117 - loss: 1.5973 - val_accuracy: 0.4448 - val_loss: 1.5399
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 19s 308ms/step - accuracy: 0.5084 - loss: 1.4239 - val_accuracy: 0.4893 - val_loss: 1.4987
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 19s 315ms/step - accuracy: 0.5634 - loss: 1.2565 - val_accuracy: 0.4847 - val_loss: 1.5225
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 332ms/step - accuracy: 0.5920 - loss: 1.1498 - val_accuracy: 0.4847 - val_loss: 1.4859
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 319ms/step - accuracy: 0.6118 - loss: 1.0728 - val_accu

In [22]:
# Evaluate the model using the testing data
model.evaluate(X_test, y_test)

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.4902 - loss: 1.5769


[1.6486166715621948, 0.49846625328063965]

In [23]:
# Saving the model
with open("initial_model.pkl", "wb") as f:
    pickle.dump(model, f)

### Testing the model

In [24]:
# reloaing the model
import pickle
with open("initial_model.pkl", "rb") as f:
    model = pickle.load(f)

In [25]:
# preprocess a new image 
import preprocessing as prep
image_path ="clothes_test_image.jpg"
X_input = prep.get_X([image_path], target_size=(150, 150))

In [27]:
import pandas as pd
output = model.predict(X_input)
pd.DataFrame(output, columns=prep.encoded_y_cols)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


,battery,cardboard,clothes,glass,metal,paper,plastic,trash
0,0.947858,0.023644,0.99977,0.560393,0.326812,0.88075,0.397096,0.000408


In [ ]:
import numpy as np
predicted_class = np.argmax(output[0])
prep.encoded_y_cols[predicted_class]

'clothes'

In [ ]:
prep.get_prediction(image_path=image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


c:\Users\Asa\miniconda3\envs\dev\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 150, 150, 3))', ())
  warnings.warn(msg)


'clothes'